In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import pymysql
import mysql.connector
import numpy as np

# List of CSV file paths
csv_files = [
    "ap_bus_details.csv", "assam_bus_details.csv", "chandigarh_bus_details.csv",
    "himachal_bus_details.csv", "kaac_bus_details.csv", "kerala_bus_details.csv",
    "rajasthan_bus_details.csv", "sb_bus_details.csv", "Telangana_bus_details.csv",
    "up_bus_details.csv", "wb_bus_details.csv", "wb2_bus_details.csv", "jk_bus_details.csv"
]

# Read each CSV file into a DataFrame and store it in a list
dfs = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

# Add an ID column
combined_df.insert(0, 'id', range(1, len(combined_df) + 1))

# Clean specific columns
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '', regex=False)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract('(\d+)')

# Count of NaN values
nan_counts = combined_df.isna().sum()
print(nan_counts)

# Fill NaN values with default values
combined_df = combined_df.fillna('default_value')

# Replace NaN values with None for MySQL compatibility
combined_df = combined_df.apply(lambda x: x.where(pd.notna(x), None))

# Save the combined DataFrame to a CSV file (optional)
combined_df.to_csv("bus_routes.csv", index=False)

# Connect to MySQL database
connection = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='Raymond@17'
)

try:
    cursor = connection.cursor()

    # Create database if it does not exist
    cursor.execute("CREATE DATABASE IF NOT EXISTS redbus")
    
    # Select the database
    cursor.execute("USE Redbus")

    # Clean column names
    combined_df.columns = combined_df.columns.str.strip()  # Remove any leading/trailing whitespace
    combined_df.columns = combined_df.columns.str.replace(' ', '_')  # Replace spaces with underscores

    # Map DataFrame dtypes to MySQL types
    dtype_mapping = {
        'float64': 'FLOAT',
        'int64': 'INT',
        'object': 'TEXT'
    }

    # Generate column definitions for the SQL query
    column_definitions = []
    for col, dtype in zip(combined_df.columns, combined_df.dtypes):
        sql_type = dtype_mapping.get(str(dtype), 'TEXT')  # Default to TEXT if dtype is unknown
        column_definitions.append(f"{col} {sql_type}")

    column_definitions_str = ", ".join(column_definitions)
    
    # Create table query
    create_table_query = f"CREATE TABLE IF NOT EXISTS bus_routes ({column_definitions_str})"
    
    # Execute table creation
    cursor.execute(create_table_query)
    
    # Prepare insert query
    insert_query = f"INSERT INTO bus_routes ({', '.join(combined_df.columns)}) VALUES ({', '.join(['%s'] * len(combined_df.columns))})"
    
    # Insert data into the table
    for row in combined_df.itertuples(index=False, name=None):
        cursor.execute(insert_query, row)
    
    # Commit the transaction
    connection.commit()

except Exception as e:
    # Print the error message
    print(f"An error occurred: {e}")
    
    # Rollback the transaction in case of an error
    connection.rollback()

finally:
    # Ensure the connection is closed
    connection.close()

id                     0
Route_Name             0
Route_Link             0
Bus_Name             308
Bus_Type             320
Departing_Time       308
Duration             308
Reaching_Time        308
Star_Rating          309
Price                308
Seat_Availability    317
dtype: int64
